#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [ ]:
#collection.delete_many({})

In [1]:
import pandas as pd
import pymongo
from pymongo import MongoClient

In [2]:
client = MongoClient("mongo") #crée le lien entre notre base Mongo et notre programme avec un `client`.
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [5]:
# on supprime les colonnes qui ne nous serviront pas
df_ks = df_ks.drop(columns=['ID', 'currency', 'deadline', 'backers'])

#on ne garde que l'année de lancement (on enlève la date et l'heure puisqu'ils ne nous serviront pas)
df_ks["launched"] = df_ks["launched"].str[:4] 

## Ici cela fonctionne mais le mieux est de changer le format en format date afin d'optimiser l'utilisation de mongodb
# df_ks["launched"] = pd.to_datetime(df_ks["launched"], errors='coerce')
# df_ks = df_ks.dropna(subset = ["launched"])

df_ks = df_ks.rename({"usd pledged":"usd_pledged"}, axis='columns')

df_ks.head()

,name,category,main_category,goal,launched,pledged,state,country,usd_pledged,usd_pledged_real
0,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,2000,2011,1145,canceled,US,1145,1145.000000
1,Ohceola jewelry,Fashion,Fashion,18000,2012,1851,failed,US,1851,1851.000000
2,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,2000,2016,7534,successful,US,3796,7534.000000
3,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,13000,2017,16298,successful,US,2670,16298.000000
4,Proven sales with custom motorcycle accessories,Sculpture,Art,5000,2016,1,failed,CA,0.708148,0.738225


### Importer les données

In [23]:
dict = df_ks.to_dict('records')

collection.insert_many(dict)

In [7]:
# verification
collection.find_one()

{'_id': ObjectId('5fc5052c7ee4a821e92e6c92'),
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'goal': 2000.0,
 'launched': '2011',
 'pledged': 1145.0,
 'state': 'canceled',
 'country': 'US',
 'usd_pledged': 1145.0,
 'usd_pledged_real': 1145.0}

## Question 1  

In [ ]:
#les 5 projets ayant reçu le plus de promesse de dons

In [8]:
# prendre les 5 pledged les plus grand

cur = collection.find().sort([("pledged",-1)]).limit(5)

list(cur)

[{'_id': ObjectId('5fc505627ee4a821e93041e6'),
  'name': 'True North ï¿½ Repurposed Sailcloth Products',
  'category': 'Product Design',
  'main_category': 'Design',
  'goal': '7500.0',
  'launched': '2016',
  'pledged': '9996.0',
  'state': 'successful',
  'country': 'US',
  'usd_pledged': '9996.0',
  'usd_pledged_real': 9996.0},
 {'_id': ObjectId('5fc505587ee4a821e92ff0c4'),
  'name': 'Scott Davis - Solo Record',
  'category': 'Music',
  'main_category': 'Music',
  'goal': '8000.0',
  'launched': '2014',
  'pledged': '9995.0',
  'state': 'undefined',
  'country': 'N,0"',
  'usd_pledged': nan,
  'usd_pledged_real': 9995.0},
 {'_id': ObjectId('5fc505547ee4a821e92fc874'),
  'name': 'Polco: A Digital Platform for Better Civic Communication',
  'category': 'Web',
  'main_category': 'Technology',
  'goal': '35000.0',
  'launched': '2017',
  'pledged': '9994.0',
  'state': 'failed',
  'country': 'US',
  'usd_pledged': '876.0',
  'usd_pledged_real': 9994.0},
 {'_id': ObjectId('5fc505517ee4a8

## Question 2

In [9]:
#nombre de projets ayant atteint leur but
#on utilise la colonne 'state' et on prends les projets 'successful'

cur = collection.count_documents({"state":"successful"})
cur

## avec une aggrégation
# cur2 = collection.aggregate([{"$group" : {"_id" : "$state", "Numberbygroup" : {"$sum" : 1}}}])
# list(cur2)

<ipython-input-9-2681e384d3b4>:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  cur_nb = cur.count()


53040


[{'_id': 'undefined', 'Numberbygroup': 1395},
 {'_id': 'canceled', 'Numberbygroup': 15359},
 {'_id': 'failed', 'Numberbygroup': 78334},
 {'_id': 'live', 'Numberbygroup': 1133},
 {'_id': '850.0', 'Numberbygroup': 1},
 {'_id': 'successful', 'Numberbygroup': 53040},
 {'_id': 'suspended', 'Numberbygroup': 738}]

## Question 3

In [10]:
#nombre de projets pour chaque catégorie

#on utilise une aggrégation
cur = collection.aggregate([{"$group" : {"_id" : "$category", "nombre" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Fashion', 'nombre': 3379},
 {'_id': 'Music Videos', 'nombre': 299},
 {'_id': 'Graphic Design', 'nombre': 765},
 {'_id': 'Narrative Film', 'nombre': 2099},
 {'_id': 'Shorts', 'nombre': 4857},
 {'_id': 'Horror', 'nombre': 525},
 {'_id': 'Installations', 'nombre': 178},
 {'_id': 'Hardware', 'nombre': 1431},
 {'_id': 'Printing', 'nombre': 83},
 {'_id': '3D Printing', 'nombre': 271},
 {'_id': 'Design', 'nombre': 1641},
 {'_id': 'Food Trucks', 'nombre': 687},
 {'_id': 'Bacon', 'nombre': 78},
 {'_id': 'Hip-Hop', 'nombre': 1555},
 {'_id': 'Jewelry', 'nombre': 472},
 {'_id': 'Crafts', 'nombre': 1834},
 {'_id': 'Cookbooks', 'nombre': 217},
 {'_id': 'Young Adult', 'nombre': 328},
 {'_id': 'Nonfiction', 'nombre': 3390},
 {'_id': 'Workshops', 'nombre': 59},
 {'_id': 'Digital Art', 'nombre': 524},
 {'_id': 'R&B', 'nombre': 172},
 {'_id': 'Grace is Leaving', 'nombre': 1},
 {'_id': 'Web', 'nombre': 2017},
 {'_id': 'Gadgets', 'nombre': 1210},
 {'_id': 'Blues', 'nombre': 113},
 {'_id': 'Romanc

## Question 4

In [26]:
# nombre de projets français ayant été instanciés avant 2016

cur = collection.count_documents({"$and":[{"country":"FR"}, {"launched":{"$lt": "2016"}}]})
cur

660

## Question 5

In [20]:
#projets américains ayant demandé plus de 200 000 dollars

cur = collection.find({"$and":[{"country":"US"}, {"goal":{"$gte": 200000}}]})

cur_nb = cur.count()
print(cur_nb)

list(cur)

<ipython-input-20-fc3eafba48aa>:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  cur_nb = cur.count()


1217


[{'_id': ObjectId('5fc5052c7ee4a821e92e6d21'),
  'name': 'Far from Par is a movie about a man and a talking golf ball.',
  'category': 'Comedy',
  'main_category': 'Film & Video',
  'goal': 200000.0,
  'launched': '2014',
  'pledged': 10.0,
  'state': 'failed',
  'country': 'US',
  'usd_pledged': 10.0,
  'usd_pledged_real': 10.0},
 {'_id': ObjectId('5fc5052c7ee4a821e92e6d34'),
  'name': 'A CALL TO ADVENTURE',
  'category': 'Film & Video',
  'main_category': 'Film & Video',
  'goal': 287000.0,
  'launched': '2012',
  'pledged': 1465.0,
  'state': 'failed',
  'country': 'US',
  'usd_pledged': 1465.0,
  'usd_pledged_real': 1465.0},
 {'_id': ObjectId('5fc5052c7ee4a821e92e6de7'),
  'name': 'Storybricks, the storytelling online RPG',
  'category': 'Video Games',
  'main_category': 'Games',
  'goal': 250000.0,
  'launched': '2012',
  'pledged': 23680.54,
  'state': 'failed',
  'country': 'US',
  'usd_pledged': 23680.54,
  'usd_pledged_real': 23680.54},
 {'_id': ObjectId('5fc5052c7ee4a821e92e6

## Question 6 

In [21]:
#le nombre de projet ayant "Sport" dans leur nom

#on utilise un index text

collection.create_index([("name",  "text")])
collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'name_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('name', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [27]:
cur = collection.count_documents( { "$text": { "$search": "Sport" } } )

cur

636